In [1]:
#imports
import numpy as np
import math
from PIL import Image, ImageOps
import io
import cv2
from star_data_generator import *
from scipy import stats
import copy
import time

In [2]:
# Cluster stars
# remove low SNR Clusters
# calculate centroids
# transform detector to sphere space
# for each star in catalog, calculate angular distance between it and every other star
# Acceptance:
#   select pole and create neighbour set
#   compute angular distances of pole and every neighbour star
#   create list of pairs from catalog with that angular distance (or close to?)
#   count number of occurrences of each star in list of pairs
#   pole star is the one that occurs most in list
#   neighbour stars are all those paired with it
#  repeat with another pole star n times
# Verification:
#   check if i-th pole is in j-th set and j-th pole in i-th set
#   if yes, check if length of intersection of accepted stars is greater than some threshold
#   if no, repeat for different j until both yes
#   if no for all j, repeat acceptance
#   if still no, use different image and start again
# Confirmation:
#   Let the pole star be confirmed
#   calculate angular displacement between confirmed star and another test star
#   build link set: stars with angular distance close to the measured one within tolerance
#   if test star is in link set, star is confirmed, add to chain set
#   if not, discard test star and try new one
#   output is final chain

In [2]:
def calculate_distance(RA_i, DE_i, RA_j, DE_j):
    return np.arccos(np.sin(DE_j)*np.sin(DE_i) + np.cos(DE_j)*np.cos(DE_i)*np.cos(RA_j-RA_i))

In [3]:
def dist1(RA_i, DE_i, RA_j, DE_j):
    return np.arccos((RA_i*RA_j+DE_i*DE_j)/(np.linalg.norm((RA_i, DE_i), 2)*np.linalg.norm((RA_j, DE_j), 2)))

In [4]:
def to_radians(degrees):
    for i in range(len(degrees)):
        if not isinstance(degrees[i], float):
            degrees[i] = to_radians(degrees[i])
        else:
            degrees[i] = math.radians(degrees[i])
    return degrees

In [24]:
# Create Feature Set
def create_feature_set():
    feature_set = [] # [(id(0), id(1)), theta < FOV]
    #dtypes = [('id_0', int), ('id_1', int), ('theta', float)]
    angle_catalog = []
    stars = len(ids)
    for i in range(stars):
        for j in range(stars):
            if i <= j:
                continue
            theta = np.abs(dist1(RAdeg[i], DEdeg[i], RAdeg[j], DEdeg[j]))
            #theta = np.abs(calculate_distance(RAdeg[i], DEdeg[i], RAdeg[j], DEdeg[j]))
            if theta <= FOV:
                feature_set.append((ids[i], ids[j], theta))
    #feature_set = np.array(feature_set, dtype=dtypes)
    #feature_set = np.sort(feature_set, order='theta')[::-1]
    return feature_set

In [6]:
#check if star is in poles list
def is_in(poles, RA, DE):
    for pole in poles:
        if pole[0] == RA and pole[1] == DE:
            return True
    return False

In [7]:
#create pole set
def get_candidate_poles(poles, a, pole_ids):
    positions = camera.to_angles(scene.pos)
    RA = positions[0]
    DE = positions[1]
    stars=[]
    for i in range(len(RA)):
        stars.append((RA[i], DE[i]))
    #still in image space here because comparing to 'middle of image'
    for i in range(len(scene.pos)):
        star = scene.pos[i]
        if star[0] < res_x-(res_x/a) and star[0] > res_x/a and star[1] < res_y-(res_y/a) and star[1] > res_y/a:
            #then compare in angle space because that is what will be used majority of time
            if not is_in(poles, RA[0], DE[0]):
                poles.append((RA[0], DE[0]))
                pole_ids.append(scene.ids[i])
        RA = RA[1:]
        DE = DE[1:]
    return poles, pole_ids

In [25]:
#Create neighbour set of first pole using angular distances

def create_distance_set(stars, pole):
    distances = []
    for star in stars:
        #distance = np.abs(dist1(star[0], star[1], pole[0], pole[1]))
        distance = np.abs(calculate_distance(star[0], star[1], pole[0], pole[1]))
        if distance != 0:
            distances.append(distance)
    return distances

In [220]:
def get_lower_pairs(features, index, distance, eps_pi):
    pairs = []
    while features[index]-distance

SyntaxError: invalid syntax (<ipython-input-220-77077de503ed>, line 3)

In [221]:
#features = np.flip(features)
stars = star_set()
distances = create_distance_set(stars, candidate_poles[0])
a = features[:100000]
b = [np.abs(np.subtract(a['theta'], distances[0]))][0]
print(np.where(b == max(b)))
print(max(b))
print(np.where(b == min(b)))
print(min(b))

(array([0]),)
0.12088175826852304
(array([54377]),)
5.523359547510154e-15


In [223]:
#Create pair list from catalog 

def create_pair_list(features, distances, eps_pi):
    start = time.time()
    pairs = []
    a = 0
    for distance in distances:
        print(a)
        a += 1
        dist_diff = [np.abs(np.subtract(i, distance)) for i in features["theta"]]
        
        #feat_copy = copy.deepcopy(features)
        #feat_copy['theta'] = [np.abs(np.subtract(i, distance)) for i in features['theta']]
        #feat_copy = np.sort(feat_copy, order='theta')
        #print(feat_copy[:2]['theta'])
        #i = 0
        #while feat_copy[i]['theta'] <= eps_pi:
        #    pairs.append((feat_copy[i]['id_0'], feat_copy[i]['id_1']))
        #    i += 1
    end = time.time()
    total_time = end - start
    print("\n"+ str(total_time))
    return pairs
#f = create_feature_set()
#pole_i, neighbours_i = acceptance_phase(f, candidate_poles[0], eps_pi)

In [124]:
#Create pair list from catalog 

#def create_pair_list(feature_set, distances, eps_pi):
#    pairs = []
#    pairs_free = []
#    step = 500

#    for distance in distances:
#        this_val = 0
#        upper_bound = -1
#        lower_bound = -1
#        pairs.append([])
#        for i in range(0, len(feature_set), step):
#            dist_diff = feature_set[i][2] - distance
#            if np.abs(dist_diff) <= eps_pi:
#                if dist_diff > 0:
#                    upper_bound = i
#                else:
#                    lower_bound = i
#            else:
#                this_val = 1
            
#            if upper_bound != -1 and this_val != 0:
#                break
#        for i in range(lower_bound, upper_bound):
#            pairs[-1].append((feature_set[i][0], feature_set[i][1]))
#            pairs_free.append((feature_set[i][0], feature_set[i][1]))
#        i = 1
#        while np.abs(feature_set[lower_bound-i][2] - distance) <= eps_pi:
#            pairs[-1].append((feature_set[lower_bound-i][0], feature_set[lower_bound-i][1]))
#            pairs_free.append((feature_set[lower_bound-i][0], feature_set[lower_bound-i][1]))
#            i += 1
#        i = 1
#        while np.abs(feature_set[upper_bound-i][2] - distance) <= eps_pi:
#            pairs[-1].append((feature_set[upper_bound-i][0], feature_set[upper_bound-i][1]))
#            pairs_free.append((feature_set[upper_bound-i][0], feature_set[upper_bound-i][1]))
#            i += 1
        #print(feature_set[lower_bound][0], feature_set[lower_bound][1], feature_set[upper_bound][0], feature_set[upper_bound][1])
        #print(lower_bound, upper_bound)
#    return pairs, pairs_free

In [9]:
def create_pair_list_long(feature_set, distances, eps_pi):
    pairs = []
    gt_count = 0
    for distance in distances:
        for i in range(len(feature_set)):
            dist_diff = feature_set[i][2] - distance
            if np.abs(dist_diff) <= eps_pi:
                pairs.append((feature_set[i][0], feature_set[i][1]))
    return pairs

In [31]:
#pole is one that occurs most in list

def find_pole_in_pairs(pair_list):
    counts = {}
    for pair in pair_list:
        for star in pair:
            if star in counts:
                counts[star] += 1
            else:
                counts[star] = 1
    max_count = max(list(counts.values()))
    #DO THIS
    position = list(counts.values()).index(max_count)
    return list(counts.keys())[position], max_count

In [11]:
# neighbours are all stars paired with pole

def create_neighbour_set(pairs, pole):
    pole_neighbours = []
    for pair in pairs:
        if pair[0] == pole:
            pole_neighbours.append(pair[1])
        elif pair[1] == pole:
            pole_neighbours.append(pair[0])
    return np.unique(np.array(pole_neighbours)).tolist()

In [12]:
#create set of stars in scene in RA,DE
def star_set():
    stars = []
    positions = camera.to_angles(scene.pos)
    RA = positions[0]
    DE = positions[1]
    for i in range(len(RA)):
        stars.append((RA[i], DE[i]))
    return stars

In [28]:
def acceptance_phase(features, candidate_pole, eps_pi):
    #a = [(np.where(ids == i)[0][0]) for i in scene.ids]
    #c_ra = np.array([[RAdeg[i] for i in a]])
    #c_de = np.array([[DEdeg[i] for i in a]])
    #confirmation = np.concatenate((c_ra.T, c_de.T), axis=1)
    #this_index = [a.index(i) for i in a if ids[i] == pole_ids[-1]][0]
    stars = star_set()
    distances = create_distance_set(stars, candidate_pole)
    #c_dist = create_distance_set(confirmation, confirmation[this_index]) #good
    ##pairs = create_pair_list(features, distances, eps_pi)
    pairs = create_pair_list_long(features, distances, eps_pi)
    print("pair list length", len(pairs))
    pole, count = find_pole_in_pairs(pairs)
    print("pole found and count", pole, count)
    neighbours = create_neighbour_set(pairs, pole)
    print("neighbour set length", len(neighbours))
    
    return pole, neighbours
#acceptance_phase(features, candidate_poles[-1], eps_pi)

In [14]:
def intersection(set_1, set_2):
    #make intersection be [(coordx, coordy), id]
    intersection = [star for star in set_1 if star in set_2]
    return intersection

In [32]:
#MPA Algorithm
start = time.time()

intersect = 0
this_pole = 0

FOV = 10
catalog = StarCatalog()
ids = catalog.catalog.HIP.values.tolist()
RAdeg = to_radians(catalog.catalog.RAdeg.tolist())
DEdeg = to_radians(catalog.catalog.DEdeg.tolist())
eps_pi = 0.0005
candidate_poles = []
pole_ids = []
reduction = 4
required_poles = 3
while len(candidate_poles) < required_poles:
    candidate_poles, pole_ids = get_candidate_poles(candidate_poles, reduction, pole_ids)
    reduction += 1

#testing stuff
#get catalog position of each star in scene
a = [(np.where(ids == i)[0][0]) for i in scene.ids]
c_ra = np.array([[RAdeg[i] for i in a]])
c_de = np.array([[DEdeg[i] for i in a]])
confirmation = np.concatenate((c_ra.T, c_de.T), axis=1)
this_index = [a.index(i) for i in a if ids[i] == pole_ids[-1]][0]


#features = create_feature_set()
#print(features.shape)
def main():
    N = len(candidate_poles)-1
    print("ground truth id", pole_ids[N])
    print("pole catalog location", np.where(ids == pole_ids[N])[0][0])
    pole_i, neighbours_i = acceptance_phase(features, candidate_poles[N], eps_pi)
    print("calculated id", pole_i)
    return
    u = False
    w = False
    j = N-1
    th = len(neighbours_i)-2
    while (u==0 or w==0) and j >= 0:
        pole_j, neighbours_j = acceptance_phase(candidate_poles[j], eps_pi)
        #print(pole_j, pole_i)
        u = (pole_i in neighbours_j) and (pole_j in neighbours_i)
        if u != 0:
            V = intersection(neighbours_i, neighbours_j)
            w = len(V) >= th
        print(len(V))
        print("pole in other set?", u, "intersection length?", w)
        j -= 1
    if u and w:
        intersect = V
        this_pole = pole_i
        confirmation_phase(V, pole_i)
        

    print(u, w, j)
    return 1

main()

end = time.time()
total_time = end - start
print("\n"+ str(total_time))

ground truth id 80953
pole catalog location 3499
pair list length 6633
pole found and count 80953 15
neighbour set length 13
calculated id 80953

197.56175327301025


In [168]:
features = create_feature_set()

In [210]:
#feat_copy = copy.deepcopy(features)
a = [(5,1,3), (4,2,-1), (3,3,0), (2,4,1), (1,5,4)]
b = np.array([[1,2,3], [4,5,6], [-1,2,3], [1,-2,3], [0,0,0]])
print(b[:,2])

#print(np.add(a,b))
#print(np.argmin(np.abs(np.add(a,b)), axis=0))
#b = np.argmin(np.abs(np.add(features, distances[0]), axis=0)

#print(min(feat_copy, key=lambda feat_copy:feat_copy[2]))

[3 6 3 3 0]


In [210]:
def confirmation_phase(intersection, pole):
    print(type(intersection))
    verified = intersection
    confirmed = pole
    print("c", confirmed)
    print("v", verified)
    displacement = np.arccos(np.dot(confirmed,verified)/(np.linalg.norm(confirmed)*np.linalg.norm(verified)))
    print(displacement)

intersect = np.array(stars[2])
this_pole = np.array(stars[1])
confirmation_phase(intersect, this_pole)

<class 'numpy.ndarray'>
c [5.11615137 1.48865144]
v [6.07441194 1.51143044]
0.039285788014714225
